<a href="https://colab.research.google.com/github/jx-dohwan/kluebert_Extract_summary/blob/main/kluebert_Extract_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 라이브러리 설치

### Mecab

In [ ]:
!pip3 install konlpy

In [ ]:
!sudo apt-get update

In [ ]:
!sudo apt-get install curl git

In [ ]:
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

### Multiprocess

In [ ]:
!pip3 install multiprocess

### Transformers

In [ ]:
!pip3 install transformers

### PyRouge

In [ ]:
!pip install pyrouge --upgrade
!pip install https://github.com/bheinzerling/pyrouge/archive/master.zip
!pip install pyrouge
!pip show pyrouge
!git clone https://github.com/andersjo/pyrouge.git
from pyrouge import Rouge155
!pyrouge_set_rouge_path '/content/pyrouge/tools/ROUGE-1.5.5'

In [ ]:
!sudo apt-get install libxml-parser-perl

In [ ]:
%%shell
cd pyrouge/tools/ROUGE-1.5.5/data
rm WordNet-2.0.exc.db # only if exist
cd WordNet-2.0-Exceptions
rm WordNet-2.0.exc.db # only if exist

./buildExeptionDB.pl . exc WordNet-2.0.exc.db
cd ../
ln -s WordNet-2.0-Exceptions/WordNet-2.0.exc.db WordNet-2.0.exc.db

### TensorboardX

In [ ]:
!pip3 install tensorboardX

### Copy Source Codes

In [ ]:
!cp -r /content/drive/MyDrive/Fast_Campus/ExSum/SRC .

## 라이브러리 Import 

In [12]:
import os
import json
from tqdm import tqdm
import re
import easydict